<a href="https://colab.research.google.com/github/LucasJFaust/bibliotecas_py/blob/main/Tunning_Regra%C3%A7%C3%A3o_Logistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('churn_data.xlsx')
df.head()

,customerID,tenure,PhoneService,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,1,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,34,Yes,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,2,Yes,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,45,No,One year,No,Bank transfer,42.30,1840.75,No
4,9237-HQITU,2,Yes,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"],errors='coerce')

In [4]:
df = df.dropna()

In [5]:
cat_vars = ['PhoneService',"Contract","PaperlessBilling","PaymentMethod"]
num_vars = ["tenure","MonthlyCharges","TotalCharges"]
modeling_vars = cat_vars + num_vars

In [6]:
X = df.filter(modeling_vars).copy()
y = df['Churn'].copy()
y = pd.DataFrame(y)

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(y.Churn)
y.Churn = le.transform(y.Churn)

In [8]:
X = pd.get_dummies(X)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [10]:
from sklearn.tree import DecisionTreeClassifier

arvore_decisao = DecisionTreeClassifier()

In [11]:
# Importamos GridSearchCV
from sklearn.model_selection import GridSearchCV

# Criamos um dicionário que os hiperparâmetros que queremos treinar o modelo
parameters = {
  'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
}

# Criamos um GridSearch passando o modelo, os parâmetros, a métrica que queremos otimizar.
grid_search = GridSearchCV(arvore_decisao, parameters, scoring='roc_auc', cv=5, n_jobs=-1)

In [12]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
             scoring='roc_auc')

In [13]:
# criando uma tabela com os resultados do grid_search
results = pd.DataFrame(grid_search.cv_results_)

# rankeando esses resultados
results.sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
4,0.022347,0.006887,0.013253,0.001344,5,{'max_depth': 5},0.830575,0.800661,0.808160,0.847528,0.816514,0.820688,0.016695,1
3,0.016258,0.000268,0.012679,0.000405,4,{'max_depth': 4},0.832061,0.796193,0.801046,0.841518,0.827125,0.819589,0.017801,2
5,0.021040,0.002120,0.011793,0.000415,6,{'max_depth': 6},0.826395,0.795995,0.795705,0.824854,0.809817,0.810553,0.013330,3
2,0.014187,0.000701,0.011894,0.000604,3,{'max_depth': 3},0.808328,0.788547,0.786743,0.829110,0.811343,0.804814,0.015725,4
6,0.021338,0.000391,0.011785,0.000457,7,{'max_depth': 7},0.814758,0.791632,0.776472,0.819436,0.786361,0.797732,0.016610,5
7,0.023374,0.001023,0.012046,0.001640,8,{'max_depth': 8},0.797130,0.772263,0.759910,0.795942,0.782331,0.781515,0.014178,6
1,0.013355,0.001922,0.012808,0.002266,2,{'max_depth': 2},0.782698,0.771267,0.778936,0.786190,0.786600,0.781138,0.005658,7
8,0.025149,0.001290,0.011830,0.000325,9,{'max_depth': 9},0.762319,0.759774,0.727506,0.767696,0.751501,0.753759,0.014128,8
9,0.027679,0.003058,0.010547,0.001588,10,{'max_depth': 10},0.742708,0.733085,0.720422,0.744014,0.728017,0.733649,0.008905,9
0,0.010067,0.001518,0.014100,0.005122,1,{'max_depth': 1},0.724878,0.717152,0.729638,0.737651,0.728643,0.727592,0.006676,10


In [14]:
from sklearn.ensemble import RandomForestClassifier

rf =  RandomForestClassifier()

In [15]:
parameters = {'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
              'n_estimators': [100, 300, 500]}

grid_search = GridSearchCV(rf, parameters, scoring='roc_auc', cv=5, n_jobs=-1)

In [16]:
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'n_estimators': [100, 300, 500]},
             scoring='roc_auc')

In [17]:
results = pd.DataFrame(grid_search.cv_results_)
results.sort_values(by='rank_test_score').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
16,2.096608,0.372069,0.096101,0.020028,6,300,"{'max_depth': 6, 'n_estimators': 300}",0.845291,0.821904,0.818027,0.856046,0.836524,0.835558,0.014207,1
19,2.173093,0.500169,0.096937,0.013303,7,300,"{'max_depth': 7, 'n_estimators': 300}",0.847409,0.820726,0.817248,0.856562,0.834983,0.835386,0.015080,2
17,2.671010,0.012426,0.140829,0.005901,6,500,"{'max_depth': 6, 'n_estimators': 500}",0.846304,0.822269,0.818025,0.854033,0.835877,0.835302,0.013715,3
20,3.060567,0.394951,0.180893,0.075918,7,500,"{'max_depth': 7, 'n_estimators': 500}",0.845814,0.820611,0.816997,0.857191,0.835683,0.835259,0.015104,4
15,0.532211,0.012728,0.037371,0.000215,6,100,"{'max_depth': 6, 'n_estimators': 100}",0.846065,0.821904,0.815691,0.855539,0.835056,0.834851,0.014755,5


In [18]:
grid_search.best_estimator_.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 6,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 300,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [19]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(n_jobs=-1)

In [20]:
from sklearn.model_selection import RandomizedSearchCV

parameters = {'learning_rate': [0.001, 0.01],
              'num_leaves': [2, 128],
              'min_child_samples': [1, 100],
              'subsample': [0.05, 1.0],
              'colsample_bytree': [0.1, 1.0]}

random_search = RandomizedSearchCV(lgbm, parameters, scoring='roc_auc', cv=5, n_iter=5, n_jobs=-1)

random_search.fit(X_train, y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1495, number of negative: 4129
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000275 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 605
[LightGBM] [Info] Number of data points in the train set: 5624, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265825 -> initscore=-1.015909
[LightGBM] [Info] Start training from score -1.015909
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


RandomizedSearchCV(cv=5, estimator=LGBMClassifier(n_jobs=-1), n_iter=5,
                   n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.1, 1.0],
                                        'learning_rate': [0.001, 0.01],
                                        'min_child_samples': [1, 100],
                                        'num_leaves': [2, 128],
                                        'subsample': [0.05, 1.0]},
                   scoring='roc_auc')

In [21]:
results = pd.DataFrame(random_search.cv_results_)
results.sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_num_leaves,param_min_child_samples,param_learning_rate,param_colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,0.845628,0.531371,0.029199,0.006164,0.05,128,100,0.001,1.0,"{'subsample': 0.05, 'num_leaves': 128, 'min_ch...",0.835560,0.807014,0.803218,0.845727,0.815328,0.821369,0.016533,1
2,0.175073,0.031715,0.027346,0.005999,0.05,128,100,0.01,0.1,"{'subsample': 0.05, 'num_leaves': 128, 'min_ch...",0.828071,0.808615,0.812407,0.832999,0.815087,0.819436,0.009422,2
4,0.317968,0.240679,0.029380,0.011650,0.05,2,100,0.001,0.1,"{'subsample': 0.05, 'num_leaves': 2, 'min_chil...",0.822376,0.807664,0.816784,0.831590,0.813321,0.818347,0.008166,3
0,0.319721,0.071609,0.032371,0.004472,0.05,128,1,0.01,0.1,"{'subsample': 0.05, 'num_leaves': 128, 'min_ch...",0.826257,0.804901,0.806087,0.825492,0.812225,0.814992,0.009230,4
1,1.042460,0.067948,0.036625,0.010298,0.05,128,1,0.001,1.0,"{'subsample': 0.05, 'num_leaves': 128, 'min_ch...",0.797266,0.802485,0.768943,0.804390,0.790117,0.792640,0.012840,5


In [22]:
from sklearn.utils.fixes import loguniform

parameters = {'learning_rate': loguniform(1e-3, 1e-1),
              'num_leaves': [2, 128],
              'min_child_samples': [1, 100],
              'subsample': [0.05, 1.0],
              'colsample_bytree': [0.1, 1.0]}

random_search = RandomizedSearchCV(lgbm, parameters, scoring='roc_auc', cv=5, n_iter=30, n_jobs=-1)

In [23]:
random_search.fit(X_train, y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1495, number of negative: 4129
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000511 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 605
[LightGBM] [Info] Number of data points in the train set: 5624, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265825 -> initscore=-1.015909
[LightGBM] [Info] Start training from score -1.015909


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


RandomizedSearchCV(cv=5, estimator=LGBMClassifier(n_jobs=-1), n_iter=30,
                   n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.1, 1.0],
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7c4d0ab37850>,
                                        'min_child_samples': [1, 100],
                                        'num_leaves': [2, 128],
                                        'subsample': [0.05, 1.0]},
                   scoring='roc_auc')

In [ ]:
results = pd.DataFrame(random_search.cv_results_)
results.sort_values(by='rank_test_score')